In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Install required packages
!pip install torch torchcrf transformers tqdm pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [12]:
# Set up paths (adjust these paths based on your actual file structure in Colab)
import os
import sys
import torch
import pandas as pd
from transformers import XLMRobertaTokenizer
from tqdm import tqdm

In [4]:
!pip install pytorch-crf

In [13]:
# Add the project root to Python path for imports
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/nlp_project_roberta"  # Change this if your files are in a different location
sys.path.append(PROJECT_DIR)

# Import the necessary modules
from dataset.dataset import get_dataloaders
from Models.xlm_roberta_model import XLMRobertaForIdiomDetection, train_model, predict_idioms, evaluate

# Define paths
DATA_DIR = os.path.join(PROJECT_DIR, "dataset")
MODELS_DIR = os.path.join(PROJECT_DIR, "Models")
OUTPUT_DIR = os.path.join(PROJECT_DIR, "Submissions")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [14]:
# Function to train the model
def run_training(epochs=10, batch_size=16, lr=2e-5, max_length=128):
    # Initialize XLM-RoBERTa tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    print("Loading data...")
    # Load and preprocess data
    train_loader, val_loader, _ = get_dataloaders(
        train_path=os.path.join(DATA_DIR, 'train.csv'),
        val_path=os.path.join(DATA_DIR, 'eval.csv'),
        batch_size=batch_size,
        max_length=max_length,
        tokenizer=tokenizer
    )

    print("Starting training...")
    # Train the model
    model = train_model(
        train_loader, val_loader, tokenizer,
        epochs=epochs, lr=lr
    )

    print('Training complete. Best model saved as best_xlm_roberta_idiom_model.pt')
    return model

In [15]:
# Function to evaluate the model
def run_evaluation(batch_size=16, max_length=128):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize XLM-RoBERTa tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    # Load data
    print("Loading evaluation data...")
    _, val_loader, _ = get_dataloaders(
        train_path=os.path.join(DATA_DIR, 'train.csv'),
        val_path=os.path.join(DATA_DIR, 'eval.csv'),
        batch_size=batch_size,
        max_length=max_length,
        tokenizer=tokenizer
    )

    # Load the model
    print("Loading model...")
    model = XLMRobertaForIdiomDetection()
    model.load_state_dict(torch.load('best_xlm_roberta_idiom_model.pt', map_location=device))
    model.to(device)

    # Evaluate the model
    print("Evaluating model...")
    metrics = evaluate(model, val_loader, tokenizer, device)
    print('Evaluation complete.')
    print(metrics)

    return metrics

In [16]:
# Function to generate predictions
def run_prediction(batch_size=16, max_length=128, output_file='prediction_xlm.csv'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize XLM-RoBERTa tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    # Load the model
    print("Loading model...")
    model = XLMRobertaForIdiomDetection()
    model.load_state_dict(torch.load('best_xlm_roberta_idiom_model.pt', map_location=device))
    model.to(device)

    # Read test data
    print("Reading test data...")
    test_df = pd.read_csv(os.path.join(DATA_DIR, 'eval_w_o_labels.csv'))
    ids = test_df['id'].tolist()
    languages = test_df['language'].tolist()
    sentences = test_df['sentence'].tolist()

    # Generate predictions
    print("Generating predictions...")
    results = []
    for idx, sentence, lang in tqdm(zip(ids, sentences, languages), total=len(ids)):
        _, idiom_indices = predict_idioms(model, tokenizer, sentence, device)
        # If no idiom, output [-1] as in training
        if not idiom_indices:
            idiom_indices = [-1]
        results.append({'id': idx, 'indices': str(idiom_indices), 'language': lang})

    # Create output dataframe and save to file
    output_path = os.path.join(OUTPUT_DIR, output_file)
    out_df = pd.DataFrame(results)
    out_df.to_csv(output_path, index=False)
    print(f'Predictions saved to {output_path}')

    return out_df

In [17]:
# Train the model (this will take time)
model = run_training(epochs=10, batch_size=16, lr=2e-5)

Loading data...
Starting training...


Epoch 1/10 [Training]: 100%|██████████| 720/720 [04:34<00:00,  2.62it/s, loss=4.4721]


Average training loss: 7.8784


Evaluating: 100%|██████████| 90/90 [00:21<00:00,  4.22it/s]


Validation loss: 4.3941
F1 Score: 1.5709
New best model saved with F1: 1.5709


...

Validation loss: 2.4070
F1 Score: 1.3202


Epoch 10/10 [Training]: 100%|██████████| 720/720 [04:34<00:00,  2.62it/s, loss=0.4422]


Average training loss: 0.6046


Evaluating: 100%|██████████| 90/90 [00:21<00:00,  4.21it/s]

Validation loss: 2.4547
F1 Score: 1.3137
Training complete. Best model saved as best_xlm_roberta_idiom_model.pt


In [18]:

# Evaluate the model
metrics = run_evaluation()

Loading evaluation data...
Loading model...
Evaluating model...


Evaluating: 100%|██████████| 90/90 [00:21<00:00,  4.23it/s]

Evaluation complete.
{'val_loss': 4.394053908189138, 'precision': 1.8304093567251463, 'recall': 1.3758241758241758, 'f1': 1.5708908406524469}


In [19]:
# Generate predictions
predictions = run_prediction(output_file='prediction_xlm.csv')

Loading model...
Reading test data...
Generating predictions...


100%|██████████| 1437/1437 [00:42<00:00, 33.83it/s]


Predictions saved to /content/drive/MyDrive/Colab Notebooks/nlp_project_roberta/Submissions/prediction_xlm.csv


In [ ]:
# Optional: Test the model with a single example
def test_single_example():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    model = XLMRobertaForIdiomDetection()
    model.load_state_dict(torch.load('best_xlm_roberta_idiom_model.pt', map_location=device))
    model.to(device)

    # Example sentence from the dataset
    sentence = "Zaman öldürmek, bir tür cinayettir, Zamanınızı öldürmeyin; zamanınızı doldurun, Kıymetini bilerek zamanınızı değerlendirin."
    processed_sentence, idiom_indices = predict_idioms(model, tokenizer, sentence, device)

    print("Sentence:", sentence)
    print("Detected Idiom Indices:", idiom_indices)

    if idiom_indices:
        words = sentence.split()
        try:
            idiom_words = [words[idx] for idx in idiom_indices]
            print("Idiom Words:", " ".join(idiom_words))
        except IndexError:
            print("Warning: Index out of range. Check tokenization.")
    else:
        print("No idiom detected.")

# Uncomment to test with a single example
# test_single_example()